In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf

In [29]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# UTILITIES

##### CONVERT NORMALISED VECTOR TO ORIGINAL FORM

In [30]:
def unNormalise(inputs, mean, std):
    return inputs*std + mean

##### LOG TO TENSORBOARD

In [48]:
def log(writer, log_dict, epoch):
    """
    log to tensorboard using the writer object
    
    @params: writer -> tf.writer object
    @params: log_dict -> (name, value) pairs, log the value using the key (name) as the tag.
    @params: epoch -> the iteration number for which logging is being done.
    """
    
    with writer.as_default():
        for metric in log_dict:
            tf.summary.scalar(metric, log_dict[metric], step=epoch)

##### LOAD DATA GIVEN RELATIVE PATH FROM data_root

In [32]:
def load(rel_path):
    """
    log to tensorboard using the writer object
    
    @params: writer -> tf.writer object
    @params: log_dict -> (name, value) pairs, log the value using the key (name) as the tag.
    @params: epoch -> the iteration number for which logging is being done.
    """
    path = os.path.join(data_root, rel_path)
    with open(path, "rb") as f:
        data = pd.read_pickle(f, compression=None)
    
    return data

##### MAKE DIRECTORY GIVEN THE PATH

In [33]:
def mkdir(*args, **kwargs):
    try:
        os.mkdir(*args, **kwargs)
    except Exception as e:
        print(e)

# INITIALISATIONS

##### PARAMETERS

In [34]:
BATCH_SIZE = 200000
EPOCHS = 10000
LR = 0.001
SCHEDULER_PAIENCE = 100
MODEL_ACTIVATION_FUNC = nn.Tanh

device = "cuda" if torch.cuda.is_available else "cpu"
print("using: ", device)

log_path = "./runs/tanh_train"
model_path = "./models/tanh_train"
graph_path = "./graphs"

data_root = "./datasets/"
data_dict = {"train": None, "val":None}

eps = 0.0000001

using:  cuda


##### INITIALISE DIRECTORIES AND LOGGERS

In [35]:
mkdir(log_path)
mkdir(model_path)
mkdir(graph_path)
    
train_writer = tf.summary.create_file_writer(os.path.join(log_path, "train"))
val_writer = tf.summary.create_file_writer(os.path.join(log_path, "val"))

[Errno 17] File exists: './runs/tanh_train'
[Errno 17] File exists: './models/tanh_train'
[Errno 17] File exists: './graphs'


# LOAD DATA

##### LOAD DATA TO data_dict

In [36]:
for file in os.listdir(data_root):
    if "train" in file: data_dict["train"] = load(file)
    if "test" in file: data_dict["val"] = load(file)

##### NORMALISE DATA

Calculate $train\_mean$ and $train\_std$. Use the same mean and standard deviation to normalise both train and test data.  
  
<i> POSSIBLE ERROR</i>

```---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-17-23cdf213d68d> in <module>
----> 1 train_mean = data_dict["train"].mean()
      2 train_std = data_dict["train"].std()
      3 
      4 data_dict["train"] = (data_dict["train"] - train_mean)/ train_std
      5 data_dict["test"] = (data_dict["test"] - train_mean)/train_std

AttributeError: 'NoneType' object has no attribute 'mean'
```
<i> REASON </i>  
Pandas tends to drop reference to data if not accessed for long time.   
  
<i> SOLUTION </i>  
Just run the cells to load data again. 



In [37]:
train_mean = data_dict["train"].mean()
train_std = data_dict["train"].std()

data_dict["train"] = (data_dict["train"] - train_mean)/ train_std
data_dict["val"] = (data_dict["val"] - train_mean)/train_std

train_mean = train_mean.to_numpy(dtype=np.float32)
train_std = train_std.to_numpy(dtype=np.float32)

In [38]:
data_dict["train"].head(10)

,m,pt,phi,eta
132784,-0.688496,-0.607629,0.868107,0.759040
99666,-0.587358,-0.612672,-1.487534,0.117474
26629,1.051897,1.503479,-1.081401,0.773105
80473,0.788036,1.697702,-0.911068,1.813972
48229,-0.578692,-0.628716,1.619709,-0.830115
61832,-0.364437,-0.492954,-1.644013,0.033356
26867,1.190307,2.021415,1.370289,-0.926956
46232,-0.641408,-0.628934,-1.075388,-1.337238
44194,-0.593362,-0.506096,1.498136,-1.235848
59782,-0.292618,-0.545608,0.744680,-1.825007


In [39]:
data_dict["val"].head(10)

,m,pt,phi,eta
85972,-0.533282,-0.581905,1.087244,-0.071133
38742,-0.472437,-0.609328,0.573286,0.476957
128711,-0.498829,-0.570312,-0.622658,0.542203
28751,0.697978,1.984290,0.178861,-1.805121
131358,-0.754120,-0.566174,0.218307,0.964189
44010,-0.535048,-0.612530,-1.351654,-0.298414
20111,-0.579275,-0.544762,0.969280,-0.823615
137856,1.133802,1.711918,1.310452,-1.261355
87065,-0.521595,-0.625987,-0.821721,1.066632
39358,-0.090846,-0.279154,-0.322752,-0.276024


##### INITIALISE PYTORCH DATALOADERS

In [40]:
train_x = torch.tensor(data_dict["train"].to_numpy(), dtype=torch.float32)
train_y = torch.tensor(data_dict["train"].to_numpy(), dtype=torch.float32)
val_x = torch.tensor(data_dict["val"].to_numpy(), dtype=torch.float32)
val_y = torch.tensor(data_dict["val"].to_numpy(), dtype=torch.float32)

train_dataset = TensorDataset(train_x, train_y)
val_dataset = TensorDataset(val_x, val_y)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=len(val_dataset))

# MODEL

##### MODEL ARCHITECTURE

<img src="assets/Architecture%20diagram.png" alt=""/>

In [41]:
class AE(nn.Module):
    def __init__(self, input_dim, encoding_dim, activation):
        super(AE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            activation(),
            nn.Linear(256, 128),
            activation(),
            nn.Linear(128, 64),
            activation(),
            nn.Linear(64, encoding_dim),
            activation(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            activation(),
            nn.Linear(64, 128),
            activation(),
            nn.Linear(128, 256),
            activation(),
            nn.Linear(256, input_dim),
        )
        
    def encode(self, inputs):
        return self.encoder(inputs)
    
    def decode(self, embeddings):
        return self.decoder(embeddings)
    
    def forward(self, inputs):
        return self.decode(self.encode(inputs))
    

##### Loss function
Loss function is the simple MSE between the decoded 4D data(from 3D encodings) and the ground truth 4D data(used to generete the 3D encoding.)

In [42]:
def Loss(pred, target):
    mseLoss = nn.MSELoss()(pred,target)
    return mseLoss

##### METRICS
These metrics are tracked for the validation set, for each step of the training.  
The system for adding new metrics is modular. New metrics can be added/tracked by simply defining a function in the following form,
<pre><code>
<b>def metric_name</b>(pred: <i>torch tensor</i>, target: <i>torch tensor</i>):
    <b>return</b> metric_value: <i>1d torch tensor</i>
</code></pre>
and then adding the function name to the metrics array.  
  
All the tracked metrics appear in tensorboard with the name of the function as label.

In [43]:
def MSE_metric(pred, target):
    return nn.MSELoss()(pred, target).cpu().detach().numpy()

def MAPE_metric(pred, target):
    std = torch.from_numpy(train_std).to(device)
    mean = torch.from_numpy(train_mean).to(device)
    return  torch.abs(std*(pred - target)/(target*std+mean+eps)).mean().cpu().detach().numpy()

In [19]:
%tensorboard --logdir={log_path} --port=12345

Reusing TensorBoard on port 12345 (pid 11678), started 0:34:09 ago. (Use '!kill 11678' to kill it.)

# TRAIN LOOP

##### INSTANTIATE MODEL, LOSS, METRICS

In [20]:
model = AE(4,3, MODEL_ACTIVATION_FUNC).to(device)
loss_func = Loss
metrics = [MAPE_metric]

##### INSTANTIATE THE OPTIMISER AND SCHEDULER
Optimiser : Adam  
Scheduler : ReduceLROnPlateau 
>Reduce Learning Rate when validation loss does not decrease for SCHEDULER_PATIENCS epochs.

In [21]:
optimiser = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, patience=SCHEDULER_PAIENCE, verbose=True, cooldown=10,factor=0.1, min_lr=1e-7)

##### TRAIN LOOP
1. The loop runs for EPOCH epochs.   
2. For each batch, after gradient decent, loss and metrics are caluclated.
3. In each epoch, the metrics and losses are calculated for both train set and val set. Actually average of metrics calculated per batch, is taken in both cases.
4. The model with the least validation loss so far is saved in model_path/best.pth and model with the latest model is stored in {model_path}/latest.pth

In [1]:
best_loss = 100000000 #the best validation loss seen so far

In [28]:
for epoch in tqdm(range(0, EPOCHS)):
    
    # aggregators for batch metrics on train set
    batch_train_losses = []
    batch_train_metrics = {metric.__name__:[] for metric in metrics}
    
    for i, (inputs, targets) in enumerate(train_loader): # for each batch
        
        # set the initial gradients to 0
        optimiser.zero_grad()
        inputs = inputs.to(device)
        target = targets.to(device)
        
        pred = model(inputs)
        loss = loss_func(pred, target)
        loss.backward()
        
        optimiser.step()
        
        # calculate all the metrics in no_grad mode to make the calculation slightly faster
        with torch.no_grad():
            batch_train_losses.append(loss.detach().cpu().numpy())
            for metric in metrics:
                name = metric.__name__
                batch_train_metrics[name].append(metric(pred, target))
    
    # the final epoch loss and metrics for the train set
    # they are caluclated by taking mean of metrics calculated for all the batches
    train_loss = np.mean(batch_train_losses)
    train_metrics = {}
    for metric in metrics:
        name = metric.__name__
        train_metrics[name] = np.mean(batch_train_metrics[name])
    
    # calculate the val loss and val metrics in no_grad mode.
    # NOTE: since validation of the entire validation is done in 1 go,
    # we don't have any per batch aggregators (ie, no batch_val_losses)
    with torch.no_grad(): 
        for val_x, val_y in val_loader:
            val_x = val_x.to(device)
            val_y = val_y.to(device)
            
            pred = model(val_x)
            val_loss = loss_func(pred, val_y).cpu().detach().numpy()
            val_metrics = {metric.__name__:metric(pred, val_y) for metric in metrics}

    # scheduler decides on whether to change the learning rate depending
    # depending in the val_losses seen for.
    scheduler.step(val_loss)
    
    # log the train loss to tensorboard
    log(train_writer, {"loss":train_loss}, epoch)
    # log the train metrics to tensorboard
    log(train_writer, train_metrics, epoch)
    
    # log the val loss to tensorboard    
    log(val_writer, {"loss":val_loss}, epoch)
    # log the val metrics to tensorboard
    log(val_writer, val_metrics, epoch)
    
    # if val loss becomes less than the best seen so far, save the model in model_path/best.pth
    if val_loss < best_loss:
        print(f"{epoch}: val_loss improved: {best_loss}->{val_loss}")
        best_loss = val_loss
        torch.save(model.state_dict(), f"{model_path}/best.pth")
    
    # save the model in model_path/latest.pth, regardless of whether imporovement happens or not.
    if epoch%10==0: torch.save(model.state_dict(), f"{model_path}/latest.pth")

/envs/cern/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


6238: val_loss improved: 0.011778169311583042->0.011771619319915771
6249: val_loss improved: 0.011771619319915771->0.01173387747257948
6250: val_loss improved: 0.01173387747257948->0.011731510050594807
6255: val_loss improved: 0.011731510050594807->0.01173141598701477
6256: val_loss improved: 0.01173141598701477->0.011725437827408314
6262: val_loss improved: 0.011725437827408314->0.011709622107446194
6263: val_loss improved: 0.011709622107446194->0.011700530536472797
6264: val_loss improved: 0.011700530536472797->0.01169855147600174
6269: val_loss improved: 0.01169855147600174->0.011691762134432793
6270: val_loss improved: 0.011691762134432793->0.01168829295784235
6271: val_loss improved: 0.01168829295784235->0.011684481985867023
6306: val_loss improved: 0.011684481985867023->0.011673849076032639
6315: val_loss improved: 0.011673849076032639->0.011660552583634853
6326: val_loss improved: 0.011660552583634853->0.011645925231277943
6327: val_loss improved: 0.011645925231277943->0.0116315

6725: val_loss improved: 0.011399933136999607->0.011399729177355766
6726: val_loss improved: 0.011399729177355766->0.011399530805647373
6727: val_loss improved: 0.011399530805647373->0.01139933429658413
6728: val_loss improved: 0.01139933429658413->0.01139913871884346
6729: val_loss improved: 0.01139913871884346->0.011398945935070515
6730: val_loss improved: 0.011398945935070515->0.011398750357329845
6731: val_loss improved: 0.011398750357329845->0.011398551985621452
6732: val_loss improved: 0.011398551985621452->0.011398355476558208
6733: val_loss improved: 0.011398355476558208->0.011398157104849815
6734: val_loss improved: 0.011398157104849815->0.011397957801818848
6735: val_loss improved: 0.011397957801818848->0.011397759430110455
6736: val_loss improved: 0.011397759430110455->0.01139756292104721
6737: val_loss improved: 0.01139756292104721->0.011397366411983967
6738: val_loss improved: 0.011397366411983967->0.011397171765565872
6739: val_loss improved: 0.011397171765565872->0.01139

6847: val_loss improved: 0.011376890353858471->0.011376708745956421
6848: val_loss improved: 0.011376708745956421->0.011376529932022095
6849: val_loss improved: 0.011376529932022095->0.011376348324120045
6850: val_loss improved: 0.011376348324120045->0.011376168578863144
6851: val_loss improved: 0.011376168578863144->0.011375987902283669
6852: val_loss improved: 0.011375987902283669->0.011375809088349342
6853: val_loss improved: 0.011375809088349342->0.011375627480447292
6854: val_loss improved: 0.011375627480447292->0.011375448666512966
6855: val_loss improved: 0.011375448666512966->0.011375268921256065
6856: val_loss improved: 0.011375268921256065->0.01137508824467659
6857: val_loss improved: 0.01137508824467659->0.011374909430742264
6858: val_loss improved: 0.011374909430742264->0.011374730616807938
6859: val_loss improved: 0.011374730616807938->0.011374549940228462
6860: val_loss improved: 0.011374549940228462->0.01137437205761671
6861: val_loss improved: 0.01137437205761671->0.011

6969: val_loss improved: 0.011355564929544926->0.011355394497513771
6970: val_loss improved: 0.011355394497513771->0.011355224996805191
6971: val_loss improved: 0.011355224996805191->0.011355055496096611
6972: val_loss improved: 0.011355055496096611->0.011354885995388031
6973: val_loss improved: 0.011354885995388031->0.011354715563356876
6974: val_loss improved: 0.011354715563356876->0.011354546993970871
6975: val_loss improved: 0.011354546993970871->0.011354376561939716
6976: val_loss improved: 0.011354376561939716->0.011354207992553711
6977: val_loss improved: 0.011354207992553711->0.01135404035449028
6978: val_loss improved: 0.01135404035449028->0.011353869922459126
6979: val_loss improved: 0.011353869922459126->0.011353700421750546
6980: val_loss improved: 0.011353700421750546->0.01135353185236454
6981: val_loss improved: 0.01135353185236454->0.011353363282978535
6982: val_loss improved: 0.011353363282978535->0.011353193782269955
6983: val_loss improved: 0.011353193782269955->0.011

...

16928: val_loss improved: 0.008775475434958935->0.008775382302701473
16929: val_loss improved: 0.008775382302701473->0.00877523235976696
16930: val_loss improved: 0.00877523235976696->0.008775042369961739
16931: val_loss improved: 0.008775042369961739->0.008775030262768269
16932: val_loss improved: 0.008775030262768269->0.008774727582931519
16934: val_loss improved: 0.008774727582931519->0.008774451911449432
16936: val_loss improved: 0.008774451911449432->0.008774283342063427
16954: val_loss improved: 0.008774283342063427->0.00877335760742426
16961: val_loss improved: 0.00877335760742426->0.008770473301410675
17003: val_loss improved: 0.008770473301410675->0.008768747560679913
17004: val_loss improved: 0.008768747560679913->0.00876549445092678
17011: val_loss improved: 0.00876549445092678->0.008764002472162247
17018: val_loss improved: 0.008764002472162247->0.008762761950492859
17020: val_loss improved: 0.008762761950492859->0.008762712590396404
17025: val_loss improved: 0.008762712590

17529: val_loss improved: 0.008690058253705502->0.00869000144302845
17531: val_loss improved: 0.00869000144302845->0.008689994923770428
17557: val_loss improved: 0.008689994923770428->0.008687537163496017
17566: val_loss improved: 0.008687537163496017->0.008685408160090446
17575: val_loss improved: 0.008685408160090446->0.008684745989739895
17582: val_loss improved: 0.008684745989739895->0.00868452899158001
17583: val_loss improved: 0.00868452899158001->0.008684426546096802
17584: val_loss improved: 0.008684426546096802->0.008682633750140667
17593: val_loss improved: 0.008682633750140667->0.00868209544569254
17594: val_loss improved: 0.00868209544569254->0.008681426756083965
17596: val_loss improved: 0.008681426756083965->0.008681005798280239
17602: val_loss improved: 0.008681005798280239->0.008680622093379498
17604: val_loss improved: 0.008680622093379498->0.008680063299834728
17606: val_loss improved: 0.008680063299834728->0.008679804392158985
17633: val_loss improved: 0.008679804392

18300: val_loss improved: 0.00859368871897459->0.008592798374593258
18307: val_loss improved: 0.008592798374593258->0.008591391146183014
18309: val_loss improved: 0.008591391146183014->0.008589712902903557
18316: val_loss improved: 0.008589712902903557->0.008588507771492004
18318: val_loss improved: 0.008588507771492004->0.008588251657783985
18324: val_loss improved: 0.008588251657783985->0.008587789721786976
18325: val_loss improved: 0.008587789721786976->0.008587214164435863
18327: val_loss improved: 0.008587214164435863->0.008586585521697998
18329: val_loss improved: 0.008586585521697998->0.008586579002439976
18333: val_loss improved: 0.008586579002439976->0.008586492389440536
18335: val_loss improved: 0.008586492389440536->0.008586258627474308
18361: val_loss improved: 0.008586258627474308->0.008585788309574127
18370: val_loss improved: 0.008585788309574127->0.008583800867199898
18372: val_loss improved: 0.008583800867199898->0.00858370866626501
18400: val_loss improved: 0.00858370

19030: val_loss improved: 0.008515564724802971->0.008515105582773685
19031: val_loss improved: 0.008515105582773685->0.008513466455042362
19043: val_loss improved: 0.008513466455042362->0.008513135835528374
19045: val_loss improved: 0.008513135835528374->0.008512021973729134
19058: val_loss improved: 0.008512021973729134->0.008511905558407307
19060: val_loss improved: 0.008511905558407307->0.00851176492869854
19062: val_loss improved: 0.00851176492869854->0.008511543273925781
19064: val_loss improved: 0.008511543273925781->0.008511236868798733
19066: val_loss improved: 0.008511236868798733->0.008510975167155266
19068: val_loss improved: 0.008510975167155266->0.008510913699865341
19101: val_loss improved: 0.008510913699865341->0.008509162813425064
19102: val_loss improved: 0.008509162813425064->0.008508936502039433
19109: val_loss improved: 0.008508936502039433->0.008508922532200813
19110: val_loss improved: 0.008508922532200813->0.008507069200277328
19118: val_loss improved: 0.00850706

19556: val_loss improved: 0.008467606268823147->0.008467554114758968
19557: val_loss improved: 0.008467554114758968->0.008465807884931564
19559: val_loss improved: 0.008465807884931564->0.008465742692351341
19562: val_loss improved: 0.008465742692351341->0.008465283550322056
19564: val_loss improved: 0.008465283550322056->0.008463302627205849
19566: val_loss improved: 0.008463302627205849->0.008462396450340748
19573: val_loss improved: 0.008462396450340748->0.008461717516183853
19578: val_loss improved: 0.008461717516183853->0.008461475372314453
19580: val_loss improved: 0.008461475372314453->0.008461297489702702
19585: val_loss improved: 0.008461297489702702->0.00846104882657528
19626: val_loss improved: 0.00846104882657528->0.00845909770578146
19627: val_loss improved: 0.00845909770578146->0.008459044620394707
19633: val_loss improved: 0.008459044620394707->0.008458980359137058
19634: val_loss improved: 0.008458980359137058->0.008457429707050323
19635: val_loss improved: 0.0084574297

# TESTING

##### LOAD BEST MODEL

In [44]:
model = AE(4,3,MODEL_ACTIVATION_FUNC)
model.to(device)
model.load_state_dict(torch.load(f"{model_path}/best.pth"))

<All keys matched successfully>

##### GENERATE THE DECODED 4D OUTPUT

In [45]:
# load val data as a torch tensor (batch size x 4)
val_data = torch.from_numpy(data_dict["val"].to_numpy(dtype=np.float32))

# pred is the predicted 4D output using the val data,
# and convert it to numpy.ndarray
pred = model(val_data.to(device)).cpu().detach().numpy()

#convert val data to a numpy.ndarray
val_data = val_data.detach().cpu().numpy()

##### UN-NORMALISE THE 4D PREDICTED OUTPUT AND 4D VALIDATION INPUT 

The model is fed val data that has been normalised (using <i> train_mean and train_std </i>).   
To get qualitative results we need to get both the predictions and val data back to the original domain.  
To get $x$ back to the original domain 
$$
x\_original = x*train\_std + train\_mean
$$

In [37]:
pred = unNormalise(pred, train_mean, train_std)
val_data = unNormalise(val_data, train_mean, train_std)

NameError: name 'pred' is not defined

##### GRAPHING FUNCTIONS

In [46]:
def makeReconstructionBar(pred, target, name, alpha=0.5):
    """
    Creates graph for the predicted 4d output and 4d val input, such that the output is 
    layerd on top of the input, with some transparency. This way we can visually see how 
    much of a deviation there for each point.
    
    target: red (background)
    pred: blue (foreground) 

    @params: pred -> predicted unnormalised values for 1 feature
    @params: target -> the ground truth unnormalised values for 1 feature
    @params: name -> name of the save file
    @params: alpha -> amount of transparency o
    """
    fig = plt.Figure(figsize=(10,10))
    ax = fig.add_subplot(1,1,1)
    
    back = target
    front = pred
    ax.bar(np.arange(back.shape[0]), back, color="r")
    ax.bar(np.arange(front.shape[0]), front, color="b", alpha=alpha)
    
    fig.savefig(name)
    plt.close()


In [47]:
def makeMAPEScatter(pred, target, name, thresh=1):
    """
    Creates a scatter graph for the mape per point for 1 feature of the val set.
    The values greater than thresh are set equal to thresh.
    
    @params: pred -> predicted unnormalised values for 1 feature
    @params: target -> the ground truth unnormalised values for 1 feature
    @params: name -> name of the save file
    @params: thresh -> the cutoff threshold.
    """
    fig = plt.Figure(figsize=(10,10))
    ax = fig.add_subplot(1,1,1)

    mape = np.abs((target-pred)/(target+0.000001))
    mask = (mape>=thresh)
    mape = mape*(1-mask) + np.ones(mape.shape)*(mask)
    ax.scatter(np.arange(mape.shape[0]), mape, color="r")
    
    fig.savefig(name)
    plt.close()

Creates a

In [110]:
def makeMAPEDistributionHist(pred, target, name, thresh=1):
    """
    Creates 2 graphs.
        
        1 histogram showing the number of points for which the MAPE lies in bins
        of 0.1, starting from 0 and till 1.
        
        1 bar graph showing the percentage of particles, in each bar of the histogram.
    
    @params: pred -> predicted unnormalised values for 1 feature
    @params: target -> the ground truth unnormalised values for 1 feature
    @params: name -> name of the save file
    @params: thresh -> the cutoff threshold.
    """
    mape = np.abs((target-pred)/(target+0.000001))
    mask = (mape>=thresh)
    mape = mape*(1-mask) + np.ones(mape.shape)*(mask)
    
    plt.hist(mape, log=False)
    plt.savefig(name)
    plt.close()
    
    plt.hist(mape, log=True)
    plt.savefig(f"log_{name}")
    plt.close()
    

In [111]:
for i in tqdm(range(val_data.shape[1])):
    makeMAPEScatter(pred[:,i], val_data[:,i], name=f"mape_scatter{i}.png")
    makeReconstructionBar(pred[:, i],val_data[:, i], name=f"reconstructed_bar{i}.png")
    makeMAPEDistributionHist(pred[:,i], val_data[:,i], name=f"mape_ditribution_hist{i}.png")
    

/envs/cern/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.
